In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

#athlete_no = '6483989' # Anna Foster
#athlete_no = '3013148' # Carolyn Birkenfeld
athlete_no = '1671873' 
url = 'https://www.athletic.net/TrackAndField/Athlete.aspx?AID='+athlete_no+'#!/L0'

response = requests.get(url)

In [5]:
chromedriver = "/usr/local/bin/chromedriver"

os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get("https://www.athletic.net")
driver.get("https://www.athletic.net/account/login/?ReturnUrl=%2Fdefault.aspx")

%load_ext dotenv
%dotenv
EMAIL = 'adds@silverbeach.com'
#PASSWORD = os.environ.get('PASSWORD')
PASSWORD = 'HappyN0w'
username_form = driver.find_element_by_name("email")
username_form.send_keys(EMAIL) # enter email

username_form.send_keys(Keys.RETURN)
password_form=driver.find_element_by_name("password") # note another approach
password_form.send_keys(PASSWORD) # enter password
password_form.send_keys(Keys.RETURN)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Using Pandas

In [21]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)

In [22]:
for df in dfs:
    if df[0][0] == '400 Meters':
        all_years = df
        
print(all_years)

              0    1           2
0    400 Meters  NaN         NaN
1  2006 Outdoor    9       52.03
2  2007 Outdoor   10     49.86 *
3  2008 Outdoor   11     49.22 *
4  2009 Outdoor   12  49.19 PR *
5   2010 Indoor   Fr    51.28 PR
6  2010 Outdoor   Fr    51.13 PR


In [13]:
dfs[2]

,0,1,2
0,400 Meters,NaN,NaN
1,2011 Outdoor,11.0,1:09.00
2,2012 Outdoor,12.0,1:08.68 PR *


### Pull data for several athletes

Loop over a list of athletes from a csv file and read their 400 Meters numbers

In [7]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [28]:
# given a time string, return a float for the time in seconds
def cleantime(time_str):
    time_str = time_str.replace('PR','')
    time_str = time_str.replace('*','')
    time_str = time_str.replace('c','')
    time_str = time_str.replace('h','')

    time_str = time_str.strip()
    #print('time = ',time_str)
    if ':' in time_str:
        t = time_str.split(':')
        time = float(t[0])*60 + float(t[1])
    elif time_str.replace('.','').isdigit(): 
        time = float(time_str)
    else: time = np.nan
    #print(time)
    return(time)

In [90]:
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athletes.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_boy1600.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_girlls 1600.csv'
file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_girl400.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_boy400.csv'

athlete_list = pd.read_csv(file)

In [91]:
# Given a list of athletes, scrap their PR times in the 400Meter race
# and add to dataframe
all_data = athlete_list[['ID','Name','School_ID','School','District']]
all_data = all_data.set_index(['ID'])
all_data['9_PR']  = np.nan
all_data['10_PR'] = np.nan
all_data['11_PR'] = np.nan
all_data['12_PR'] = np.nan
all_data['Grad_Yr'] = 0
all_data.Grad_Yr = all_data.Grad_Yr.astype(int)

In [78]:
all_data.head()

,Name,School_ID,School,District,9_PR,10_PR,11_PR,12_PR,Grad_Yr
ID,,,,,,,,,
133440,Li Murphy,302,Annie Wright,District 9,NaN,NaN,NaN,NaN,0
1675244,Olivia Thomas,345,King's,District 8,NaN,NaN,NaN,NaN,0
216722,Whitney Leavitt,377,Kiona-Benton,District 5,NaN,NaN,NaN,NaN,0
298973,Elise Wilcox,303,Bush,District 9,NaN,NaN,NaN,NaN,0
319625,Clara Midgley,308,University Prep,District 9,NaN,NaN,NaN,NaN,0


In [92]:
all_data.shape

(7094, 9)

In [93]:
#all_data1 = all_data[0:2000]
all_data2 = all_data[2000:3000]
all_data3 = all_data[3000:5000]
all_data4 = all_data[5000:8000]
all_data5 = all_data[8000:10000]
print(all_data1.shape,all_data2.shape,all_data3.shape,all_data4.shape,all_data5.shape)

(2000, 10) (1000, 9) (2000, 9) (2094, 9) (0, 9)


In [73]:
all_data = all_data1

In [74]:
all_data.head()

,Name,School_ID,School,District,9_PR,10_PR,11_PR,12_PR,Grad_Yr
ID,,,,,,,,,
133440,Li Murphy,302,Annie Wright,District 9,NaN,NaN,NaN,NaN,0
1675244,Olivia Thomas,345,King's,District 8,NaN,NaN,NaN,NaN,0
216722,Whitney Leavitt,377,Kiona-Benton,District 5,NaN,NaN,NaN,NaN,0
298973,Elise Wilcox,303,Bush,District 9,NaN,NaN,NaN,NaN,0
319625,Clara Midgley,308,University Prep,District 9,NaN,NaN,NaN,NaN,0


In [99]:
all_data = all_data4

Event = '400 Meters'
#Event = '1600 Meters'
n = 0
for ID, row in all_data.iterrows():
    ID_str = str(ID)
    url = 'https://www.athletic.net/TrackAndField/Athlete.aspx?AID='+ID_str+'#!/L0'
    r = requests.get(url, headers=header)
    dfs = pd.read_html(r.text)
    for df in dfs:
        if df[0][0] == Event:
            n += 1
            if n%100 == 0: print(n)

            df.drop(df.index[0:1],inplace=True)
            for index, row in df.iterrows():
                if 'Indoor' not in str(row[0]):
                    #print(f"grade {int(row[1])}, time {row[2]}")
                    #row[1]= str(row[1]).replace('-','')
                    g = str(row[1])
                    g = g.replace('-','')
                    g = g.replace('.0','')

                    #print(ID_str,g)
                    if (g == '9' or g == '10' or g == '11' or g == '12'): 
                        #grade = str(int(row[1]))
                        grade = g

                    else:
                        grade = '0'
                    time = cleantime(str(row[2]))
                    #print(grade,time)
                    all_data.at[ID,grade+"_PR"] = time
# find the graduation year
                    if g == '12': 
                        gy = str(row[0])
                        gy = gy.replace(' Outdoor','')
                        all_data.at[ID,'Grad_Yr'] = int(gy)
                        #print(int(gy))


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [100]:
all_data4 = all_data

In [96]:
all_data2.head()

,Name,School_ID,School,District,9_PR,10_PR,11_PR,12_PR,Grad_Yr,0_PR
ID,,,,,,,,,,
3011282,Jessica Miller,22531,Glacier Peak,District 8,69.54,64.64,65.35,NaN,0,NaN
347193,Chelsea Dowdell,475,Everett,District 8,NaN,NaN,NaN,69.57,2011,NaN
2953009,Alex Laiblin,485,Oak Harbor,District 8,69.92,NaN,67.24,64.52,2014,NaN
2961675,Jazzmin Carroll,475,Everett,District 8,NaN,70.34,NaN,NaN,0,NaN
2929728,Marcail McDonald,481,Meadowdale,District 8,NaN,NaN,NaN,70.93,2011,NaN


In [55]:
all_data

,Name,School_ID,School,District,9_PR,10_PR,11_PR,12_PR,Grad_Yr
ID,,,,,,,,,
11047561,Luke Absher,358,Port Townsend,District 3,79.64,63.95,57.63,54.63,2018
5315996,Randy Johnson,336,Okanogan,District 6,58.88,58.02,59.22,57.94,2018


In [101]:
all_data = pd.concat([all_data1,all_data2,all_data3,all_data4])
#all_data = pd.concat([all_data1,all_data2])

In [ ]:
all_data.columns

In [102]:
#all_data.drop(['8_PR'], axis=1,inplace=True)
#all_data.drop(['7_PR'], axis=1,inplace=True)
all_data.drop(['0_PR'], axis=1,inplace=True)

In [105]:
all_data.dropna(inplace=True)
all_data.dropna().shape

(382, 9)

In [106]:
all_data.to_csv("allDistrict_girl400.csv")

In [103]:
all_data.shape

(7094, 9)

In [107]:
all_data

,Name,School_ID,School,District,9_PR,10_PR,11_PR,12_PR,Grad_Yr
ID,,,,,,,,,
133440,Li Murphy,302,Annie Wright,District 9,58.45,57.50,57.43,57.54,2010
1675244,Olivia Thomas,345,King's,District 8,62.35,59.78,59.19,58.98,2009
319625,Clara Midgley,308,University Prep,District 9,60.36,61.07,60.07,61.88,2011
184213,Mackenna Moore,310,Cascade Christian,District 3,64.61,64.86,61.49,62.79,2010
243672,Stephanie Rice,345,King's,District 8,64.43,63.34,64.58,68.43,2011
296046,Alia Payne,360,Vashon Island,District 3,67.14,64.92,67.44,67.12,2011
354829,Maddie Knaub,383,Chelan,District 6,69.20,65.26,65.24,65.24,2011
216714,Emily Blizard,377,Kiona-Benton,District 5,65.91,63.10,65.28,63.34,2010
251433,Courtney Hornby,367,Rochester,District 4,66.01,65.34,66.24,66.94,2011


all_data1.shape